# Random City Model
_Felix Haumann_

Prerequisites:
- Excel file with city map where Roads are marked with "X"
- Excel map needs to have one lot marked with "D" which is the depot

Model parameters:
- Every road accessible lot will have a house
- House have between 1 and 15 households
- Households are assigned to the houses at (insert probability here)
- Every household can potentially receive parcels
- Probability for a household is (insert probability here)
- The probability is calculated for each household within a house and the resulting parcels then summed up for that house

Model limitations:
- Houses have no orientation so parcels can be delivered to the 'backside'

---
# SETUP

### Library imports

In [495]:
import copy
import math
import random
from typing import Tuple, Dict, Any, List

import xlwings as xw
import pandas as pd
import numpy as np
import time
import ast
import csv


from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
from sklearn.cluster import KMeans
from IPython.display import display
import string
from pprint import pprint
from helper import generate_letter_combinations, save_df_as_csv, read_df_from_csv, elapsed_time, export_map_to_excel_with_formatting
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


## Helper functions

In [496]:
def read_excel_map_as_df(wb_path, ws_name, col_range, skiprows, number_of_rows, number_of_cols):
    col_names = list(range(number_of_cols))
    print("read_excel_map_as_df:",col_range, skiprows, "number_of_rows", number_of_rows, "number_of_cols", number_of_cols)
    df = pd.read_excel(wb_path, sheet_name=ws_name, usecols=col_range, skiprows=skiprows, nrows=number_of_rows, names=col_names)
    print(df)
    return df

In [497]:
def get_lot_xy_from_lot_name(ref_dict: dict, lot_name:str):
    return ref_dict[lot_name][key_h_xy]

In [498]:
def write_master_house_dict_to_csv(data, filename):

    # Define the fieldnames for the CSV file
    fieldnames = ["lot",
                  key_h_xy,
                  key_h_lot_use,
                  key_h_households,
                  key_h_parcels,
                  key_h_delivery_district,
                  key_h_path_finding_val,
                  key_h_delivery_runs_required,
                  key_h_parcels_redirected

                  # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                  ]

    # Write the dictionary to the CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for key, value in data.items():
            writer.writerow({"lot": key,
                             key_h_xy: value[key_h_xy],
                             key_h_lot_use: value[key_h_lot_use],
                             key_h_households: value[key_h_households],
                             key_h_parcels: value[key_h_parcels],
                             key_h_delivery_district: value[key_h_delivery_district],
                             key_h_path_finding_val: value[key_h_path_finding_val],
                             key_h_delivery_runs_required: value[key_h_delivery_runs_required],
                             key_h_parcels_redirected: value[key_h_parcels_redirected]

                             # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                             })

def read_csv_to_master_house_dict(filename):
    temp_dict = {}

    # Read the CSV file and populate the dictionary
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            lot = row["lot"]
            xy = tuple(map(int, row[key_h_xy].strip("()").split(",")))
            lot_use = row[key_h_lot_use]
            households = int(row[key_h_households])
            parcels = int(row[key_h_parcels])
            delivery_district = row[key_h_delivery_district]
            path_finding_val = int(row[key_h_path_finding_val])
            delivery_runs_required = int(row[key_h_delivery_runs_required])
            parcels_redirected = int(row[key_h_parcels_redirected])

            # INSERT NEW ENTRIES ABOVE
            temp_dict[lot] = {key_h_xy: xy,
                              key_h_lot_use: lot_use,
                              key_h_households: households,
                              key_h_parcels: parcels,
                              key_h_delivery_district: delivery_district,
                              key_h_path_finding_val: path_finding_val,
                              key_h_delivery_runs_required: delivery_runs_required,
                              key_h_parcels_redirected: parcels_redirected

                              # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                              }
    return temp_dict

In [499]:
def write_or_read_depots(depots_xy: list, path: str, read_or_write: str):
    if read_or_write == "w":
        # Writing to CSV file
        with open(path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(depots_xy)
    elif read_or_write == "r":
        # Reading from CSV file
        with open(path, mode='r') as file:
            reader = csv.reader(file)
            depots_xy_read = [(int(row[0]), int(row[1])) for row in reader]
        return depots_xy_read

In [500]:
def get_excel_column_name(col_index):
    """
    Returns the Excel column name (e.g., 'A', 'B', ..., 'Z', 'AA', 'AB', ..., 'ZZ', 'AAA', ...)
    for a given column index.
    """
    col_name = ''
    while col_index > 0:
        col_index, remainder = divmod(col_index - 1, 26)
        col_name = chr(65 + remainder) + col_name
    return col_name

## Model parameters

### Switches
rerun_simulation: decides whether the simulation will be run again or whether the available csv files will be used

In [501]:
rerun_simulation = True

### Paths

In [502]:
# Excel files
blank_wb_city = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\CITY_BLANK.xlsx"
blank_ws_city = "BLANK"

households_wb_city = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\CITY_HOUSEHOLDS.xlsx"
households_ws_city = "HOUSEHOLDS"
households_col_range = "C:CX"

quantity_wb_city = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\CITY_QTY.xlsx"
quantity_ws_city = "QTY"

excel_pathvis_wb = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\PathVisualisation_TEST.xlsx"
excel_pathvis_ws = "VIS"

# CSV filenames
# TODO: add check if csv files are all available
csv_name_map_base = "map_base"
csv_distance_matrix_base = "distance_matrix_district_"

master_house_csv_path = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\csv_files\master_houses_dict.csv"
depots_xy_csv_path = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\csv_files\depots_xy.csv"



### City parameters

In [503]:
# map_size_x = 20
# map_size_y = 20




# Defining the map boundaries from the Excel file
first_row = 3
first_col = 3
# KISS = Keep it nice and SQUARE
last_row = 42
last_col = 42
num_of_rows = 30
num_of_cols = 40

column_range = f"{get_excel_column_name(first_col)}:{get_excel_column_name(first_col+num_of_cols-1)}"

print(column_range)
column_range = "C:AP"
skiprows = 1
map_char_street_read = "X"  # Map char which is read from the Excel template
map_char_depot = "D"
map_char_parcel = "P"
map_char_house = "H"
map_char_empty_lot = "."
map_char_street_write = " "  # Map char which will be printed to the visual outputs
name_string_depot = "DEPOT"


delivery_districts={"A":[(0,0),(19,4)],
                    "B":[(0,5),(19,9)],
                    "C":[(0,10),(19,14)],
                    "D":[(0,15),(19,19)],
                    "E":[(20,0),(39,4)],
                    "F":[(20,5),(39,9)],
                    "G":[(20,10),(39,14)],
                    "H":[(20,15),(39,19)],
                    "I":[(0,20),(39,29)]
                    }




# # Defining the map boundaries from the Excel file
# first_row = 3
# first_col = 3
# # KISS = Keep it nice and SQUARE
# last_row = 42
# last_col = 42
# num_of_rows = last_row-first_row+1
# num_of_cols = last_col-first_col+1
# column_range = "C:AP"
# skiprows = 1
# map_char_street = "X"
# map_char_depot = "D"
# name_string_depot = "DEPOT"
#
# delivery_districts={"A":[(0,0),(19,4)],
#                     "B":[(0,5),(19,9)],
#                     "C":[(0,10),(19,14)],
#                     "D":[(0,15),(19,19)],
#                     "E":[(20,0),(39,4)],
#                     "F":[(20,5),(39,9)],
#                     "G":[(20,10),(39,14)],
#                     "H":[(20,15),(39,19)],
#                     "I":[(0,20),(39,39)]
#                     }


# # Defining the map boundaries from the Excel file
# first_row = 3
# first_col = 3
# last_row = 102
# last_col = 102
# num_of_rows = last_row-first_row+1
# num_of_cols = last_col-first_col+1
# column_range = "C:CX"
# skiprows = 1
# map_char_street = "X"
# map_char_depot = "D"
# name_string_depot = "DEPOT"
#
# # DEBUG Switch back on
# delivery_districts = {"A": [(0, 0), (33, 23)],
#                       "B": [(34, 0), (71, 26)],
#                       "C": [(72, 0), (99, 35)],
#                       "D": [(0, 24), (34, 47)],
#                       "E": [(35, 27), (71, 52)],
#                       "F": [(72, 36), (99, 66)],
#                       "G": [(0, 48), (34, 71)],
#                       "H": [(35, 53), (71, 75)],
#                       "I": [(72, 67), (99, 99)],
#                       "J": [(0, 72), (34, 99)],
#                       "K": [(35, 76), (71, 99)]
#                       }

# Pathfinding variables
# These variables are assigned to the cells for pathfinding
# Tiles with val 1 = walkable
# Tiles with val >1 = walkable but higher cost
# Tiles with val 0 or negative = not walkable
# Note: the last tile (end_coordinates) must have a positive value otherwise it cant be reached
path_val_depot = 200
path_val_house = 500
path_val_house_with_parcels = 100
path_val_empty_lot = 0
path_val_road = 1

C:AP


### Multiplier parameters

In [504]:
par_sys_distance_multiplier = 0.75  # Modifying the distances, effectively modding the map scale

### System parameters

#### Conventional delivery

In [505]:
par_sys_conv_type_name = "conventional"
par_sys_conv_vehicle_speed = 10

#### Autonomous delivery

In [506]:
par_sys_aut_type_name = "autonomous"
par_sys_aut_vehicle_capacity = 1
par_sys_aut_vehicle_speed = 6

### Weights & distributions

In [507]:
# Parcel quantity choices and weights
parcel_quantity_choices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
parcel_quantity_weights = [0.8996, 0.0634, 0.0233, 0.0085, 0.0031, 0.0011, 0.0004, 0.0001, 0.0001, 0.0001, 0.0001]

# Household size choices and weights
household_size_choices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
household_size_weights = [0.4, 0.35, 0.045, 0.04, 0.035, 0.03, 0.025, 0.02, 0.015, 0.01, 0.005, 0.0025, 0.0125, 0.005, 0.005]

# Parcel delivery redirection rate
parcel_redirected_choices = [False, True]
parcel_redirected_weights = {par_sys_conv_type_name: [0.95, 0.05], par_sys_aut_type_name: [0.97, 0.03]}

---
# Central datastructures

## Master_Houses_Dict

In [508]:
master_houses_dict = {}

# IMPORTANT: CHANGES TO DICT STRUCTURE REQUIRE INTEGRATION INTO IMPORT AND EXPORT FUNCTION

key_h_xy = "xy"
key_h_lot_use = "lot_use"
key_h_path_finding_val = "path_finding_val"
key_h_households = "households"
key_h_parcels = "parcels"
key_h_delivery_district = "delivery_district"
key_h_delivery_runs_required = "delivery_runs_required"  # Depending on carry capacity autonomous vehicles might have to drive to house more often
key_h_parcels_redirected = "parcels_redirected"  # Parcels which could not be delivered and were returned to the depot
# Final format will be: {"AAA":{"xy":(0,0), "lot_use":"P", "households":2, "parcels":4, "deliver_district":"A", "delivery_runs_required":4}, "AAB":{"xy":(1,0), "lot_use": ".", "households":0, "parcels":0, "delivery_district":"", "delivery_runs_required":0, "redirected_parcels":1}}

In [509]:
def populate_master_houses_dict(height:int = num_of_rows, width:int = num_of_cols):
    letter_combinations = generate_letter_combinations()
    print("width", width)
    print("height", height)
    for y in range(width):
        for x in range(height):
            name = next(letter_combinations)
            master_houses_dict[name] = {key_h_xy: (x, y), key_h_lot_use: None, key_h_households: 0, key_h_parcels: 0, key_h_delivery_district: "", key_h_path_finding_val:0, key_h_delivery_runs_required:0, key_h_parcels_redirected:0}

populate_master_houses_dict()

width 40
height 30


In [510]:
pprint(master_houses_dict)

{'AAA': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (0, 0)},
 'AAB': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (1, 0)},
 'AAC': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (2, 0)},
 'AAD': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (3, 0)},
 'AAE': {'delivery_district': '',
         'delivery

## Master_Routes_Dict

In [511]:
# Dict that will contain all the information about the conventional delivery districts

# Final format will be:
# {<districtA>: {"paths":{<vehicle_id1>: ['DEPOT1', 'ATI', 'AXC', 'BMM']}, "path_lengths" : {<vehicle_id1>: 450}},
# <districtB>: {"paths":{<vehicle_id1>: ['DEPOT1', 'ATI', 'AXC', 'BMM']}, "path_lengths" : {<vehicle_id1>: 450}}}

master_routes_dict = {}

key_r_paths = "paths"
key_r_path_lengths = "path_lengths"


## Master_Autonomous_Routes_Dict

In [512]:
# Dict that will contain all the information about the autonomous delivery systems

# Final format will be:
# {<districtA>: {"distance": 1550, "further_parameter": "something"}},
# <districtB>:{"distance": 2540, "further_parameter": "something_else"}}

master_autonomous_dict = {}

key_a_distance = "distance"

## Master_Model_Output_Dict

In [513]:
# Dict that will contain all the return information of the model

# Final format will be:
# {<districtA>: {"parcels": 180, "parcels_redirected": 4, "stops": 110, "distance_conventional": 853, "distance_autonomous":1759, "vehicles_used_autonomous":15}},
# <districtB>: {"parcels": 158, "parcels_redirected": 1, "stops": 120, "distance_conventional": 965, "distance_autonomous":2864, "vehicles_used_autonomous":15}}

master_model_output_dict = {}

key_o_parcels = "parcels"
key_o_stops = "stops"
key_o_drop_factor = "drop_factor"
key_o_distance_conventional = "dist_conv"
key_o_distance_autonomous = "dist_auto"
key_o_vehicles_autonomous = "vehicles_auto"
key_o_parcels_redirected = "parcels_redirected"

## Master_City_Dict

In [514]:
# Dict that will contain all the general information about the city

# Final format will be:
# {"houses": 567, "households":1759, "total_parcels":156, "total_parcels_redirected": 14, "total_stops": 110}

master_city_dict = {}

key_c_houses = "houses"
key_c_households = "households"
key_c_total_parcels = "total_parcels"
key_c_total_parcels_redirected = "total_parcels_redirected"
key_c_total_stops = "total_stops"

---
# Prepare maps

### Base map from Excel

In [515]:
if rerun_simulation:
    # DEBUG
    # map_base = read_excel_map_as_df(wb_path=blank_wb_city, ws_name=blank_ws_city, col_range=column_range,
    #                                 skiprows=skiprows, number_of_rows=num_of_rows, number_of_cols=num_of_cols)
    map_base = read_excel_map_as_df(wb_path=blank_wb_city, ws_name=blank_ws_city, col_range=column_range,
                                    skiprows=skiprows, number_of_rows=num_of_rows, number_of_cols=num_of_cols)
    save_df_as_csv(map_base, csv_name_map_base)
else:
    map_base = read_df_from_csv(csv_name_map_base)

pd.set_option('display.max_columns', None, 'display.max_rows', None)
display(map_base)



read_excel_map_as_df: C:AP 1 number_of_rows 30 number_of_cols 40
     0    1    2    3    4    5    6    7    8    9  10   11   12   13   14   
0   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X    X    X    X    X  \
1   NaN    D  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
2   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
3   NaN    X    X    X    X    X    X    X    X    X  X    X    X    X    X   
4   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
5   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
6   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
7   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
8   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN  NaN  NaN   
9   NaN    X    X    X    X    X    X    X    X    X  X    X    X    X    X   
10  NaN    X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  X  NaN  NaN 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
2,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
3,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X
4,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
5,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
6,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
7,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
8,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
9,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X


### Copy base map

In [516]:
map_lot_use = map_base.copy()
map_lot_use = map_lot_use.fillna(0)  # Replace NaN with empty lots

In [517]:
map_parcel = map_base.copy()

In [518]:
map_pathfinding = map_base.copy()

In [519]:
map_lot_names = map_base.copy()

### Map styling

In [520]:
def display_styled_map(map_to_style):
    center_style = [
        {'selector': 'td',
         'props': [('text-align', 'center')]}
    ]
    styled_map = map_to_style.style
    styled_map.set_table_styles(center_style)
    display(styled_map)

---
# Populate city with houses

In [521]:
def lot_next_to_road(map_df: pd.DataFrame, x:int, y:int, map_char_street) -> bool:
    """
    Checks if current lot is next to a road
    """
    # OG version
    # own = map_df.iat[y, x]
    # above = map_df.iat[y - 1, x] if y - 1 >= 0 else 0
    # left = map_df.iat[y, x - 1] if x - 1 >= 0 else 0
    # below = map_df.iat[y + 1, x] if y + 1 < map_df.shape[0] else 0
    # right = map_df.iat[y, x + 1] if x + 1 < map_df.shape[1] else 0

    own = map_df.iat[x, y]
    above = map_df.iat[x - 1, y] if x - 1 >= 0 else 0
    left = map_df.iat[x, y - 1] if y - 1 >= 0 else 0
    below = map_df.iat[x + 1, y] if x + 1 < map_df.shape[0] else 0
    right = map_df.iat[x, y + 1] if y + 1 < map_df.shape[1] else 0



    #print("own",own, "above",above, "left", left, "right", right, "below",below)  # Debug


    around = {"a": True if above == map_char_street else False,
              "l": True if left == map_char_street else False,
              "r": True if right == map_char_street else False,
              "b": True if below == map_char_street else False}
    any_surround = any(around.values())

    if own != map_char_street and any_surround:
        return True
    else:
        return False


In [522]:
def select_household_size():
    result = random.choices(household_size_choices, household_size_weights, k=1)
    res_int = result[0]
    return res_int

In [523]:
def select_parcel_quantity():
    result = random.choices(parcel_quantity_choices, parcel_quantity_weights, k=1)
    res_int = result[0]
    return res_int

In [524]:
def parcels_were_redirected(system_type: str):
    """
    Returns True if parcels for household were redirected else False
    :param system_type: "conventional" | "autonomous"
    :return:
    """
    result = random.choices(parcel_redirected_choices, parcel_redirected_weights[system_type], k=1)
    res_bool = result[0]
    return res_bool

In [525]:
pprint(master_houses_dict)

{'AAA': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (0, 0)},
 'AAB': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (1, 0)},
 'AAC': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (2, 0)},
 'AAD': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': None,
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (3, 0)},
 'AAE': {'delivery_district': '',
         'delivery

In [526]:
def evaluate_lot_use(map_df: pd.DataFrame) -> list:
    """
    Evaluates every lot in the master_dict for it use.
    If a lot is by a road a house will be places, the households in the house chosen at random based on the choices and weights defined.
    The parcels each household receives which are again based in choices and weights are summed up for each house.
    The master_dict will be edited to incorporate the lot use for printing a pretty map.
    The coordinates of the depot or depots will be returned as list of tuples.
    :param map_df:
    :return: depot_coordinates
    """
    depot_coordinates = []
    i = 0
    for lot, info in master_houses_dict.items():
        print(f"i = {i}")
        i+=1
        x, y = info[key_h_xy]
        print("X", x, "y", y)
        # print("lot", lot)
        cell_val = map_df.iat[x,y] # DEBUG war im og [y,x]
        print(f"{lot}[{y},{x}]{cell_val}, [{x},{y}]{map_df.iat[x, y]}")
        # print(cell_val)
        if cell_val == 0:
            if lot_next_to_road(map_df, x, y, map_char_street_read):
                households_in_house = select_household_size()
                parcels_to_house = 0
                parcels_to_house_redirected = 0
                for household in range(households_in_house):
                    parcels_for_household = select_parcel_quantity()
                    if parcels_for_household > 0 and parcels_were_redirected(par_sys_conv_type_name):
                        parcels_to_house_redirected += parcels_for_household
                        print(f"Redirected {parcels_for_household} parcels. Total: {parcels_to_house_redirected} parcels redirected")
                    parcels_to_house += parcels_for_household
                info[key_h_households] = households_in_house
                info[key_h_parcels] = parcels_to_house
                info[key_h_parcels_redirected] = parcels_to_house_redirected
                if parcels_to_house > 0:
                    info[key_h_lot_use] = map_char_parcel
                    info[key_h_path_finding_val] = path_val_house_with_parcels
                else:
                    info[key_h_lot_use] = map_char_house
                    info[key_h_path_finding_val] = path_val_house
            else:
                info[key_h_lot_use] = map_char_empty_lot
        elif cell_val == map_char_depot:
            info[key_h_lot_use] = map_char_depot
            info[key_h_path_finding_val] = path_val_depot
            depot_coordinates.append(info[key_h_xy])
        elif cell_val == map_char_street_read:
            info[key_h_lot_use] = map_char_street_write
            info[key_h_path_finding_val] = path_val_road
        else:
            info[key_h_lot_use] = "WTF"
    return depot_coordinates


In [527]:
if rerun_simulation:
    depots_xy = evaluate_lot_use(map_lot_use)
    write_or_read_depots(depots_xy, depots_xy_csv_path, "w")
    write_master_house_dict_to_csv(master_houses_dict, master_house_csv_path)
    pprint(master_houses_dict)
    print(f"Depot locations: {depots_xy}")
    print("------")
    print("Lot use evaluated")
else:
    print("Reading lot use from csv...")
    depots_xy = write_or_read_depots([], depots_xy_csv_path, "r")
    master_houses_dict = read_csv_to_master_house_dict(master_house_csv_path)
    pprint(master_houses_dict)
    print(f"Depot locations: {depots_xy}")
    print("------")
    print("Lot use read from csv")


i = 0
X 0 y 0
AAA[0,0]0, [0,0]0
i = 1
X 1 y 0
AAB[0,1]0, [1,0]0
i = 2
X 2 y 0
AAC[0,2]0, [2,0]0
i = 3
X 3 y 0
AAD[0,3]0, [3,0]0
i = 4
X 4 y 0
AAE[0,4]0, [4,0]0
i = 5
X 5 y 0
AAF[0,5]0, [5,0]0
i = 6
X 6 y 0
AAG[0,6]0, [6,0]0
i = 7
X 7 y 0
AAH[0,7]0, [7,0]0
i = 8
X 8 y 0
AAI[0,8]0, [8,0]0
i = 9
X 9 y 0
AAJ[0,9]0, [9,0]0
i = 10
X 10 y 0
AAK[0,10]0, [10,0]0
i = 11
X 11 y 0
AAL[0,11]0, [11,0]0
i = 12
X 12 y 0
AAM[0,12]0, [12,0]0
i = 13
X 13 y 0
AAN[0,13]0, [13,0]0
i = 14
X 14 y 0
AAO[0,14]0, [14,0]0
Redirected 2 parcels. Total: 2 parcels redirected
i = 15
X 15 y 0
AAP[0,15]0, [15,0]0
i = 16
X 16 y 0
AAQ[0,16]0, [16,0]0
i = 17
X 17 y 0
AAR[0,17]0, [17,0]0
i = 18
X 18 y 0
AAS[0,18]0, [18,0]0
i = 19
X 19 y 0
AAT[0,19]0, [19,0]0
i = 20
X 20 y 0
AAU[0,20]0, [20,0]0
i = 21
X 21 y 0
AAV[0,21]0, [21,0]0
i = 22
X 22 y 0
AAW[0,22]0, [22,0]0
i = 23
X 23 y 0
AAX[0,23]0, [23,0]0
i = 24
X 24 y 0
AAY[0,24]0, [24,0]0
i = 25
X 25 y 0
AAZ[0,25]0, [25,0]0
i = 26
X 26 y 0
ABA[0,26]0, [26,0]0
i = 27
X 27 y 0
AB

In [528]:
pprint(master_houses_dict)


{'AAA': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': '.',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (0, 0)},
 'AAB': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': '.',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (1, 0)},
 'AAC': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 5,
         'lot_use': 'H',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 500,
         'xy': (2, 0)},
 'AAD': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 1,
         'lot_use': 'H',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 500,
         'xy': (3, 0)},
 'AAE': {'delivery_district': '',
         'delivery

---
# Populate maps

## Show parcel locations on map

In [529]:
def populate_parcel_map():
    for name, info in master_houses_dict.items():
        x,y = info[key_h_xy]
        lot_use = info[key_h_lot_use]
        map_parcel.iat[x, y] = lot_use #DEBUG was [y,x]

In [530]:
populate_parcel_map()
display_styled_map(map_parcel)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.,.,.,.,.,.,.,.,.,H,,,,,,,,,,,,,,,,H,.,.,.,.,H,.,.,.,.,.,H,.,.,.
1,.,D,.,.,.,.,.,.,.,H,,H,H,P,H,H,P,H,H,H,H,H,P,H,,H,.,.,.,P,,P,.,.,.,P,,H,.,.
2,H,,H,H,P,H,H,P,H,P,,P,P,H,H,H,H,P,P,H,H,.,.,P,,P,P,H,H,P,,H,H,H,P,P,,H,H,H
3,H,,,,,,,,,,,,,,,,,,,,,H,.,P,,,,,,,,,,,,,,,,
4,H,,P,H,H,P,P,H,H,P,,H,P,H,H,H,H,H,H,H,,H,.,H,,H,H,P,H,H,,H,H,H,P,H,,H,P,H
5,H,,H,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,P,,H,.,P,,P,.,.,.,H,,H,.,.,.,H,,P,.,.
6,H,,P,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,H,,H,.,H,,H,.,.,.,H,,P,.,.,.,H,,H,.,.
7,P,,H,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,P,,H,.,H,,H,.,.,.,H,,H,.,.,.,H,,P,.,.
8,H,,H,P,H,H,H,H,H,H,,H,H,P,H,H,H,H,P,P,,H,H,H,,H,H,H,H,P,,H,P,H,H,P,,H,H,H
9,H,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## Populate lot names map

In [531]:
def populate_lot_names_map():
    for name, info in master_houses_dict.items():
        x,y = info[key_h_xy]
        lot_use = info[key_h_lot_use]
        print_name = name if lot_use not in [".", " "] else ""
        lot_use = lot_use if lot_use in ["."] else ""
        map_lot_names.iat[x, y] = f"{print_name}{lot_use}"  # DEBUG was [y,x]

In [532]:
populate_lot_names_map()
display_styled_map(map_lot_names)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.,.,.,.,.,.,.,.,.,AKK,,,,,,,,,,,,,,,,BCW,.,.,.,.,BIQ,.,.,.,.,.,BPO,.,.,.
1,.,ABF,.,.,.,.,.,.,.,AKL,,AMT,ANX,APB,AQF,ARJ,ASN,ATR,AUV,AVZ,AXD,AYH,AZL,BAP,,BCX,.,.,.,BHN,,BJV,.,.,.,BOL,,BQT,.,.
2,AAC,,ACK,ADO,AES,AFW,AHA,AIE,AJI,AKM,,AMU,ANY,APC,AQG,ARK,ASO,ATS,AUW,AWA,AXE,.,.,BAQ,,BCY,BEC,BFG,BGK,BHO,,BJW,BLA,BME,BNI,BOM,,BQU,BRY,BTC
3,AAD,,,,,,,,,,,,,,,,,,,,,AYJ,.,BAR,,,,,,,,,,,,,,,,
4,AAE,,ACM,ADQ,AEU,AFY,AHC,AIG,AJK,AKO,,AMW,AOA,APE,AQI,ARM,ASQ,ATU,AUY,AWC,,AYK,.,BAS,,BDA,BEE,BFI,BGM,BHQ,,BJY,BLC,BMG,BNK,BOO,,BQW,BSA,BTE
5,AAF,,ACN,.,.,.,.,.,.,AKP,,AMX,.,.,.,.,.,.,.,AWD,,AYL,.,BAT,,BDB,.,.,.,BHR,,BJZ,.,.,.,BOP,,BQX,.,.
6,AAG,,ACO,.,.,.,.,.,.,AKQ,,AMY,.,.,.,.,.,.,.,AWE,,AYM,.,BAU,,BDC,.,.,.,BHS,,BKA,.,.,.,BOQ,,BQY,.,.
7,AAH,,ACP,.,.,.,.,.,.,AKR,,AMZ,.,.,.,.,.,.,.,AWF,,AYN,.,BAV,,BDD,.,.,.,BHT,,BKB,.,.,.,BOR,,BQZ,.,.
8,AAI,,ACQ,ADU,AEY,AGC,AHG,AIK,AJO,AKS,,ANA,AOE,API,AQM,ARQ,ASU,ATY,AVC,AWG,,AYO,AZS,BAW,,BDE,BEI,BFM,BGQ,BHU,,BKC,BLG,BMK,BNO,BOS,,BRA,BSE,BTI
9,AAJ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


---
# Pathfinding

## Setup Pathfinding

#### Transform map for pathfinding

In [533]:
map_pathfinding = map_pathfinding.replace(map_char_street_read, -999)  # Replace 'X' values with -999 --> Road
map_pathfinding = map_pathfinding.replace(map_char_depot, -1000)  # Replace 'D' values with -1000 --> Depot
map_pathfinding = map_pathfinding.fillna(path_val_empty_lot) # Replace NaN values with -2 --> Empty lots
map_pathfinding = map_pathfinding.apply(pd.to_numeric, errors='coerce', downcast='integer')  # Transform float values to ints
map_pathfinding = map_pathfinding.mask(map_pathfinding >= 0, -1)  # Replace positive numeric values with -1
map_pathfinding = map_pathfinding.replace(-999, path_val_road)  # Replace -999 values with road_val
map_pathfinding = map_pathfinding.replace(-1000, path_val_depot)  # Replace -1000 values with depot_val

map_pathfinding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,200,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
2,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
3,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
5,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
6,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
7,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
8,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1
9,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### Populate pathfinding map with values from master_dict

In [534]:
def populate_pathfinding_map():
    for lot, info in master_houses_dict.items():
        x, y = info[key_h_xy]
        path_value = info[key_h_path_finding_val]
        map_pathfinding.iat[x,y] = path_value  # Debug was [y,x]

In [535]:
populate_pathfinding_map()
display(map_pathfinding)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0,0,0,0,0,0,0,0,0,500,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,500,0,0,0,0,500,0,0,0,0,0,500,0,0,0
1,0,200,0,0,0,0,0,0,0,500,1,500,500,100,500,500,100,500,500,500,500,500,100,500,1,500,0,0,0,100,1,100,0,0,0,100,1,500,0,0
2,500,1,500,500,100,500,500,100,500,100,1,100,100,500,500,500,500,100,100,500,500,0,0,100,1,100,100,500,500,100,1,500,500,500,100,100,1,500,500,500
3,500,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,500,0,100,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,500,1,100,500,500,100,100,500,500,100,1,500,100,500,500,500,500,500,500,500,1,500,0,500,1,500,500,100,500,500,1,500,500,500,100,500,1,500,100,500
5,500,1,500,0,0,0,0,0,0,500,1,500,0,0,0,0,0,0,0,100,1,500,0,100,1,100,0,0,0,500,1,500,0,0,0,500,1,100,0,0
6,500,1,100,0,0,0,0,0,0,500,1,500,0,0,0,0,0,0,0,500,1,500,0,500,1,500,0,0,0,500,1,100,0,0,0,500,1,500,0,0
7,100,1,500,0,0,0,0,0,0,500,1,500,0,0,0,0,0,0,0,100,1,500,0,500,1,500,0,0,0,500,1,500,0,0,0,500,1,100,0,0
8,500,1,500,100,500,500,500,500,500,500,1,500,500,100,500,500,500,500,100,100,1,500,500,500,1,500,500,500,500,100,1,500,100,500,500,100,1,500,500,500
9,500,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [536]:
def instantiate_grid(map_df: pd.DataFrame, export_matrix:bool = False):
    pathfinding_map_matrix = map_df.values.tolist()
    grid_export = Grid(matrix=pathfinding_map_matrix)
    if export_matrix:
        return grid_export, pathfinding_map_matrix
    else:
        return grid_export

In [537]:
def find_path(grid: Grid, start_coords: tuple, end_coords: tuple):
    """
    Finds path on a provided grid between two nodes

    :param grid:
    :param start_coords:
    :param end_coords:
    :return: path_res, path_len_res
    """
    start_y, start_x  = start_coords  # Debug Was start_x, start_y
    end_y, end_x  = end_coords  # Debug was end_x, end_y
    start = grid.node(start_x, start_y)
    end = grid.node(end_x, end_y)
    finder = AStarFinder(diagonal_movement=DiagonalMovement.never)
    path_res, runs = finder.find_path(start, end, grid)
    path_len_res = len(path_res)
    # print('operations:', runs, 'path length:', path_len)
    # print(grid.grid_str(path=path, start=start, end=end))
    grid.cleanup()
    return path_res, path_len_res

In [538]:
def draw_path_to_map(path_to_draw: list, map_to_print: pd.DataFrame):
    path_marker = "🟥"
    print_map = map_to_print.copy()
    # FixMe: if first or last node is above or below the start or finish pop it of because it will otherwise overlap the stop or start
    #path_to_draw.pop(0)
    #path_to_draw.pop(len(path_to_draw)-1)
    for loc in path_to_draw:
        y, x = loc  # Debug was x,y # no idea why it is flipped here
        print(loc)
        print_map.iat[x,y] = path_marker    # Debug was [y,x]
    return print_map

In [539]:
def draw_from_lot_to_lot(start_lot: str|tuple, end_lot: str|tuple):
    if type(start_lot) is str:
        start_lot = master_houses_dict[start_lot][key_h_xy]
    if type(end_lot) is str:
        end_lot = master_houses_dict[end_lot][key_h_xy]

    path, path_len = find_path(map_pathfinding_grid, start_lot, end_lot)
    display_styled_map(draw_path_to_map(path, map_parcel))


## Execute pathfinding

In [540]:
map_pathfinding_grid, map_pathfinding_matrix = instantiate_grid(map_pathfinding, export_matrix=True)

In [541]:
# Draw path from depot to random parcel location for testing
def choose_far_away_node():
    master_houses_dict_as_list = list(master_houses_dict.items())
    # Iterate in reverse using reversed() and a for loop
    for lot_name, info in reversed(master_houses_dict_as_list):
        if info[key_h_lot_use] == map_char_parcel:
            print(f"Found {lot_name} with {info[key_h_xy]}")
            return tuple(info[key_h_xy])

debug_coord_s = (1,1)
debug_coord_e = choose_far_away_node()

path, path_len = find_path(map_pathfinding_grid, debug_coord_s, debug_coord_e)
display_styled_map(draw_path_to_map(path, map_parcel))


Found BUC with (28, 39)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(2, 9)
(3, 9)
(4, 9)
(5, 9)
(6, 9)
(7, 9)
(8, 9)
(9, 9)
(10, 9)
(11, 9)
(12, 9)
(13, 9)
(14, 9)
(15, 9)
(16, 9)
(17, 9)
(18, 9)
(19, 9)
(20, 9)
(21, 9)
(22, 9)
(23, 9)
(23, 10)
(23, 11)
(23, 12)
(23, 13)
(23, 14)
(24, 14)
(25, 14)
(26, 14)
(27, 14)
(28, 14)
(28, 15)
(28, 16)
(28, 17)
(28, 18)
(28, 19)
(29, 19)
(30, 19)
(31, 19)
(32, 19)
(33, 19)
(34, 19)
(34, 20)
(34, 21)
(34, 22)
(34, 23)
(34, 24)
(34, 25)
(34, 26)
(34, 27)
(35, 27)
(36, 27)
(37, 27)
(38, 27)
(39, 27)
(39, 28)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.,.,.,.,.,.,.,.,.,H,,,,,,,,,,,,,,,,H,.,.,.,.,H,.,.,.,.,.,H,.,.,.
1,.,🟥,.,.,.,.,.,.,.,H,,H,H,P,H,H,P,H,H,H,H,H,P,H,,H,.,.,.,P,,P,.,.,.,P,,H,.,.
2,H,🟥,H,H,P,H,H,P,H,P,,P,P,H,H,H,H,P,P,H,H,.,.,P,,P,P,H,H,P,,H,H,H,P,P,,H,H,H
3,H,🟥,,,,,,,,,,,,,,,,,,,,H,.,P,,,,,,,,,,,,,,,,
4,H,🟥,P,H,H,P,P,H,H,P,,H,P,H,H,H,H,H,H,H,,H,.,H,,H,H,P,H,H,,H,H,H,P,H,,H,P,H
5,H,🟥,H,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,P,,H,.,P,,P,.,.,.,H,,H,.,.,.,H,,P,.,.
6,H,🟥,P,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,H,,H,.,H,,H,.,.,.,H,,P,.,.,.,H,,H,.,.
7,P,🟥,H,.,.,.,.,.,.,H,,H,.,.,.,.,.,.,.,P,,H,.,H,,H,.,.,.,H,,H,.,.,.,H,,P,.,.
8,H,🟥,H,P,H,H,H,H,H,H,,H,H,P,H,H,H,H,P,P,,H,H,H,,H,H,H,H,P,,H,P,H,H,P,,H,H,H
9,H,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,🟥,,,,,,,,,,,,,,,,


---
# Create district routes

## Split parcel locations into delivery districts

In [542]:
def determine_delivery_district(coords):
    """
    Evaluates provided coordinates to return the delivery district it belongs to
    :param coords:
    :return:
    """
    y, x = coords # Debug was x, y
    for district, coords_range in delivery_districts.items():
        x_min, y_min = coords_range[0]
        x_max, y_max = coords_range[1]
        if x_min <= x <= x_max and y_min <= y <= y_max:
            return district
    raise ValueError

In [543]:
def evaluate_lots_for_delivery_district():
    """
    Checks which lot with a parcel quantity of > 0 belongs into which delivery district
    """
    for lot, info in master_houses_dict.items():
        print(lot, info)
        if info[key_h_parcels]>0:
            info[key_h_delivery_district] = determine_delivery_district(info[key_h_xy])

In [544]:
evaluate_lots_for_delivery_district()

AAA {'xy': (0, 0), 'lot_use': '.', 'households': 0, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 0, 'delivery_runs_required': 0, 'parcels_redirected': 0}
AAB {'xy': (1, 0), 'lot_use': '.', 'households': 0, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 0, 'delivery_runs_required': 0, 'parcels_redirected': 0}
AAC {'xy': (2, 0), 'lot_use': 'H', 'households': 5, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 500, 'delivery_runs_required': 0, 'parcels_redirected': 0}
AAD {'xy': (3, 0), 'lot_use': 'H', 'households': 1, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 500, 'delivery_runs_required': 0, 'parcels_redirected': 0}
AAE {'xy': (4, 0), 'lot_use': 'H', 'households': 2, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 500, 'delivery_runs_required': 0, 'parcels_redirected': 0}
AAF {'xy': (5, 0), 'lot_use': 'H', 'households': 1, 'parcels': 0, 'delivery_district': '', 'path_finding_val': 500, 'delivery_runs_required': 0, 'parcels_

In [545]:
pprint(master_houses_dict)

{'AAA': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': '.',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (0, 0)},
 'AAB': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 0,
         'lot_use': '.',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 0,
         'xy': (1, 0)},
 'AAC': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 5,
         'lot_use': 'H',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 500,
         'xy': (2, 0)},
 'AAD': {'delivery_district': '',
         'delivery_runs_required': 0,
         'households': 1,
         'lot_use': 'H',
         'parcels': 0,
         'parcels_redirected': 0,
         'path_finding_val': 500,
         'xy': (3, 0)},
 'AAE': {'delivery_district': '',
         'delivery

In [546]:
def split_parcel_locations_into_districts() -> dict:
    """
    Splits all nodes that have parcels into a dict with the delivery district as key.
    The depot(s) are added to each district
    :return: {"A":{"AAB":(1,3),"AAD":(4,5)}, "B":{"DAF":(8,4),....}}
    """
    district_quantities = {}
    for district in delivery_districts.keys():
        district_nodes = {}
        for index, depot_coordinates in enumerate(depots_xy):
            depot_name = f"{name_string_depot}{index+1}"
            district_nodes[depot_name] = depot_coordinates
        for lot, info in master_houses_dict.items():
            if info[key_h_delivery_district] == district:
                district_nodes[lot] = info[key_h_xy]
        district_quantities[district] = district_nodes
    return district_quantities

In [547]:
nodes_per_district = split_parcel_locations_into_districts()

In [548]:
# DEBUG
for node in nodes_per_district["A"]:
    if name_string_depot not in node:
        print(master_houses_dict[node])

{'xy': (4, 2), 'lot_use': 'P', 'households': 1, 'parcels': 1, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 0}
{'xy': (2, 4), 'lot_use': 'P', 'households': 2, 'parcels': 3, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 0}
{'xy': (4, 5), 'lot_use': 'P', 'households': 15, 'parcels': 6, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 3}
{'xy': (4, 6), 'lot_use': 'P', 'households': 2, 'parcels': 1, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 0}
{'xy': (2, 7), 'lot_use': 'P', 'households': 4, 'parcels': 3, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 0}
{'xy': (2, 9), 'lot_use': 'P', 'households': 2, 'parcels': 1, 'delivery_district': 'A', 'path_finding_val': 100, 'delivery_runs_required': 0, 'parcels_redirected': 

In [549]:
pprint(nodes_per_district)

{'A': {'ACM': (4, 2),
       'AES': (2, 4),
       'AFY': (4, 5),
       'AHC': (4, 6),
       'AIE': (2, 7),
       'AKM': (2, 9),
       'AKO': (4, 9),
       'AMU': (2, 11),
       'ANY': (2, 12),
       'AOA': (4, 12),
       'APB': (1, 13),
       'ASN': (1, 16),
       'ATS': (2, 17),
       'AUW': (2, 18),
       'DEPOT1': (1, 1)},
 'B': {'AAH': (7, 0),
       'ACO': (6, 2),
       'ADU': (8, 3),
       'API': (8, 13),
       'AVC': (8, 18),
       'AWD': (5, 19),
       'AWF': (7, 19),
       'AWG': (8, 19),
       'DEPOT1': (1, 1)},
 'C': {'AAK': (10, 0),
       'AAO': (14, 0),
       'AFE': (14, 4),
       'AIM': (10, 7),
       'AKU': (10, 9),
       'ANE': (12, 11),
       'AOJ': (13, 12),
       'APK': (10, 13),
       'AUA': (10, 17),
       'AWI': (10, 19),
       'AWJ': (11, 19),
       'DEPOT1': (1, 1)},
 'D': {'AAP': (15, 0),
       'AAT': (19, 0),
       'AFF': (15, 4),
       'AGN': (19, 5),
       'AHR': (19, 6),
       'AIT': (17, 7),
       'AJX': (17, 8),
      

## Generate distance matrices within districts

In [550]:
def instantiate_distance_matrices(nodes_in_district: dict, importing_matrices: bool = False) -> dict:
    """
    Creates individual (blank) distance matrices for all delivery districts.
    :param nodes_in_district:
    :param importing_matrices: Flag whether the blank distance matrices will be generated.
    :return: distance_matrices_dict
    {"A": <DF with all lots with parcels in district A>,
    "B": <DF with all lots with parcels in district A>}
    """
    distance_matrices_dict = {}
    if not importing_matrices:
        for district, nodes in nodes_in_district.items():
            lot_names = list(nodes.keys())
            distance_matrix = pd.DataFrame(index=lot_names, columns=lot_names)
            distance_matrices_dict[district] = distance_matrix
    return distance_matrices_dict

In [551]:
instantiate_distance_matrices(nodes_per_district)

{'A':        DEPOT1  ACM  AES  AFY  AHC  AIE  AKM  AKO  AMU  ANY  AOA  APB  ASN   
 DEPOT1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  \
 ACM       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AES       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AFY       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AHC       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AIE       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AKM       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AKO       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AMU       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 ANY       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 AOA       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
 APB       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

In [552]:
def get_remaining_stops(stops: dict, cur_key:str) -> dict:
    """
    Evaluates the remaining nodes which the distance has not been calculated for yet for the current key.
    :param stops:
    :param cur_key:
    :return:
    """
    remaining_stops = {}
    found_key = False
    for key, value in stops.items():
        if key == cur_key:
            found_key = True
        elif found_key:
            remaining_stops[key] = value
    return remaining_stops

In [553]:
def populate_distance_matrix(nodes_in_district: dict, dst_matrix: pd.DataFrame, district: str) -> pd.DataFrame:
    """
    Calculates the distances between each node
    :param dict nodes_in_district: Dict of all lots that will get parcels {'LHU': (38, 76), 'LHZ': (43, 76), 'LIF': (49, 76)}
    :param pd.DataFrame dst_matrix: DF of the instantiated distance matrix
    :param district
    :return:
    """
    return_df = dst_matrix
    for lot_a, node_a in nodes_in_district.items():
        print("------")
        remaining_stops = get_remaining_stops(stops=nodes_in_district, cur_key=lot_a).items()
        remaining_nodes = len(dict(remaining_stops).keys())
        print(f"Remaining nodes: {remaining_nodes}")
        print(f"Checking distances for: {lot_a}{node_a}")
        for lot_b, node_b in remaining_stops:
            _, distance = find_path(grid=map_pathfinding_grid,start_coords=node_a,end_coords=node_b)
            #print(f"[{district}] Distance from {lot_a}{node_a} to {lot_b}{node_b} = {distance} units")
            return_df.loc[lot_a, lot_b] = distance
            return_df.loc[lot_b, lot_a] = distance

    return_df = return_df.fillna(0)  # Fill distance between itself with 0
    return return_df

In [554]:
print(nodes_per_district)

{'A': {'DEPOT1': (1, 1), 'ACM': (4, 2), 'AES': (2, 4), 'AFY': (4, 5), 'AHC': (4, 6), 'AIE': (2, 7), 'AKM': (2, 9), 'AKO': (4, 9), 'AMU': (2, 11), 'ANY': (2, 12), 'AOA': (4, 12), 'APB': (1, 13), 'ASN': (1, 16), 'ATS': (2, 17), 'AUW': (2, 18)}, 'B': {'DEPOT1': (1, 1), 'AAH': (7, 0), 'ACO': (6, 2), 'ADU': (8, 3), 'API': (8, 13), 'AVC': (8, 18), 'AWD': (5, 19), 'AWF': (7, 19), 'AWG': (8, 19)}, 'C': {'DEPOT1': (1, 1), 'AAK': (10, 0), 'AAO': (14, 0), 'AFE': (14, 4), 'AIM': (10, 7), 'AKU': (10, 9), 'ANE': (12, 11), 'AOJ': (13, 12), 'APK': (10, 13), 'AUA': (10, 17), 'AWI': (10, 19), 'AWJ': (11, 19)}, 'D': {'DEPOT1': (1, 1), 'AAP': (15, 0), 'AAT': (19, 0), 'AFF': (15, 4), 'AGN': (19, 5), 'AHR': (19, 6), 'AIT': (17, 7), 'AJX': (17, 8), 'ANH': (15, 11), 'ANJ': (17, 11), 'ANL': (19, 11), 'APT': (19, 13), 'AQX': (19, 14), 'ATB': (15, 16), 'ATF': (19, 16), 'AUF': (15, 17), 'AWQ': (18, 19)}, 'E': {'DEPOT1': (1, 1), 'AZL': (1, 22), 'BAQ': (2, 23), 'BAR': (3, 23), 'BCY': (2, 25), 'BEC': (2, 26), 'BFI':

In [555]:
# Populate distance matrices
if rerun_simulation:
    start_time = time.time()
    distance_matrices = instantiate_distance_matrices(nodes_in_district=nodes_per_district, importing_matrices=False)
    for district in delivery_districts.keys():
        print(f"Current district: {district}")
        tmp_df = populate_distance_matrix(nodes_per_district[district], distance_matrices[district], district)
        distance_matrices[district] = tmp_df
        print(tmp_df)
        save_file_name = f"{csv_distance_matrix_base}{district}"
        save_df_as_csv(df_to_save=tmp_df, csv_name_no_filetype=save_file_name, with_header_and_rows=True)
        print(f"Saved '{save_file_name}'.")
    pprint(distance_matrices)
    elapsed_time(start_time)
else:

    distance_matrices = instantiate_distance_matrices(nodes_in_district=nodes_per_district, importing_matrices=False)
    pprint(distance_matrices)
    column_names_dict = {}
    for district, distance_matrix in distance_matrices.items():
        column_names_dict[district] = distance_matrix.columns.tolist()

    #print("column_names_dict",column_names_dict)

    for district in delivery_districts.keys():
        #print(f"Column Names for district {district}: {column_names_dict[district]}")
        renamed_columns = {}
        for index, lot_name in enumerate(column_names_dict[district]):
            renamed_columns[index] = lot_name
        print(renamed_columns)
        print(f"Retrieving data from csv for district {district}")
        tmp_df = read_df_from_csv(f"{csv_distance_matrix_base}{district}", with_header_and_rows=True)
        #tmp_df = tmp_df.rename(columns=renamed_columns)
        distance_matrices[district] = tmp_df
        del tmp_df


    pprint(distance_matrices["A"]) #Debug

Current district: A
------
Remaining nodes: 14
Checking distances for: DEPOT1(1, 1)
------
Remaining nodes: 13
Checking distances for: ACM(4, 2)
------
Remaining nodes: 12
Checking distances for: AES(2, 4)
------
Remaining nodes: 11
Checking distances for: AFY(4, 5)
------
Remaining nodes: 10
Checking distances for: AHC(4, 6)
------
Remaining nodes: 9
Checking distances for: AIE(2, 7)
------
Remaining nodes: 8
Checking distances for: AKM(2, 9)
------
Remaining nodes: 7
Checking distances for: AKO(4, 9)
------
Remaining nodes: 6
Checking distances for: AMU(2, 11)
------
Remaining nodes: 5
Checking distances for: ANY(2, 12)
------
Remaining nodes: 4
Checking distances for: AOA(4, 12)
------
Remaining nodes: 3
Checking distances for: APB(1, 13)
------
Remaining nodes: 2
Checking distances for: ASN(1, 16)
------
Remaining nodes: 1
Checking distances for: ATS(2, 17)
------
Remaining nodes: 0
Checking distances for: AUW(2, 18)
        DEPOT1  ACM  AES  AFY  AHC  AIE  AKM  AKO  AMU  ANY  AOA 

In [556]:
display(distance_matrices)

{'A':         DEPOT1  ACM  AES  AFY  AHC  AIE  AKM  AKO  AMU  ANY  AOA  APB  ASN   
 DEPOT1       0    5    7    8    9   10   12   12   14   15   15   19   22  \
 ACM          5    0    5    6    7    8   10   10   12   13   13   17   20   
 AES          7    5    0    4    5    6    8    8   10   11   11   15   18   
 AFY          8    6    4    0    2    5    7    7    9   10   10   14   17   
 AHC          9    7    5    2    0    4    6    6    8    9    9   13   16   
 AIE         10    8    6    5    4    0    5    5    7    8    8   12   15   
 AKM         12   10    8    7    6    5    0    3    3    6    6    8   11   
 AKO         12   10    8    7    6    5    3    0    5    6    6   10   13   
 AMU         14   12   10    9    8    7    3    5    0    2    4    8   11   
 ANY         15   13   11   10    9    8    6    6    2    0    3   11   14   
 AOA         15   13   11   10    9    8    6    6    4    3    0   11   14   
 APB         19   17   15   14   13   12    8  

## Create delivery routes

### Setup data model

In [557]:
vrp_data = {}
vrp_data_key_distance_matrix = "distance_matrix"
vrp_data_key_num_vehicles = "num_vehicles"
vrp_data_key_depot = "depot"

In [558]:
def prepare_data_model(distance_matrix: dict, number_of_vehicles: int = 1, depot_index: int = 0) -> dict:
    """
    Returns dictionary with distance matrix as np array, number of vehicles for the route calculating, and the index of the depot in the np array
    """
    vrp_data = {vrp_data_key_distance_matrix: distance_matrix.to_numpy(),
                vrp_data_key_num_vehicles: number_of_vehicles,
                vrp_data_key_depot: depot_index}
    return vrp_data


### Prepare data output

In [559]:
def output_solution(data, manager, routing, solution):
    """
    Prints solution to console.
    :param data:
    :param manager:
    :param routing:
    :param solution:
    :return:
    """
    #print(f"Objective: {solution.ObjectiveValue()}")
    route_paths = {}
    route_distances = {}
    optimal_route_per_vehicle = {}
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        print_routes = ""
        route_distance = 0
        optimal_route = []
        while not routing.IsEnd(index):
            route_index = manager.IndexToNode(index)
            print_routes += f" {route_index} -> "
            optimal_route.append(route_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        print_routes += f"{manager.IndexToNode(index)}"
        optimal_route.append(manager.IndexToNode(index))  # Add index of depot as last node
        route_distances[vehicle_id] = route_distance
        route_paths[vehicle_id] = print_routes
        optimal_route_per_vehicle[vehicle_id] = optimal_route
    return route_paths, optimal_route_per_vehicle, route_distances

In [560]:
def extract_lot_names_from_route_indices(route_indices_per_vehicle: dict, distance_matrix: pd.DataFrame):
    lot_names = distance_matrix.columns.tolist()
    all_routes_with_lot_names_per_vehicle = {}
    for vehicle_id, route in route_indices_per_vehicle.items():
        route_with_lot_names = []
        for node_index in route:
            route_with_lot_names.append(lot_names[node_index])
        all_routes_with_lot_names_per_vehicle[vehicle_id]=route_with_lot_names
    return all_routes_with_lot_names_per_vehicle

### Instantiate model

In [561]:
def evaluate_vrp(distance_matrix: dict, number_of_vehicles: int = 1) -> tuple[dict, dict, dict]:
    """
    Evaluates the vrp for one delivery district for each set delivery vehicle.
    :param distance_matrix:
    :param number_of_vehicles:
    :return: print_routes, route_indices, path_distances
    """
    # Prepare data model
    data = prepare_data_model(distance_matrix=distance_matrix,
                              number_of_vehicles=number_of_vehicles,
                              depot_index=0)
    # Setup index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'],
                                           data['depot'])
    # Instantiate routing model
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Calculate cost of each path
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add distance constraint
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        5000,  # vehicle maximum travel distance
        True,  # start cumulate to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    print_route_per_vehicle = {}
    route_indices_per_vehicle = {}
    path_distance_per_vehicle={}
    # Print solution on console.
    if solution:
        print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle = output_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

    return print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle


### Execute model for each district

In [562]:
def find_routes_for_delivery_districts(delivery_districts: dict, distance_matrices: dict):
    if rerun_simulation:
        print("Analysing delivery routes")
        routes_per_district = {}
        for district in delivery_districts:
            print(f"Analyzing delivery district: {district}")
            relevant_distance_matrix = distance_matrices[district]
            print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle = evaluate_vrp(relevant_distance_matrix)
            routes_per_vehicle = extract_lot_names_from_route_indices(route_indices_per_vehicle, relevant_distance_matrix)
            for vehicle_id in routes_per_vehicle:
                print(f"Optimized route for vehicle {vehicle_id} in district {district}:")
                print(" -> ".join(routes_per_vehicle[vehicle_id]))
                print(f"Total distance for vehicle {vehicle_id} in district {district}: {path_distance_per_vehicle[vehicle_id]} units")
                print(f"----------")
            routes_per_district[district] = {}
            routes_per_district[district]["paths"] = routes_per_vehicle
            routes_per_district[district]["path_lengths"] = path_distance_per_vehicle
        print("Saving delivery routes to csv")
        df_routes_per_district = pd.DataFrame(routes_per_district)
        save_df_as_csv(df_routes_per_district, "vehicle_delivery_routes", True)

    else:
        print("Reading delivery routes from csv files")
        df_routes_per_district = read_df_from_csv("vehicle_delivery_routes", True)
    return df_routes_per_district


In [563]:
routes_per_district = find_routes_for_delivery_districts(delivery_districts=delivery_districts, distance_matrices=distance_matrices)

Analysing delivery routes
Analyzing delivery district: A
Optimized route for vehicle 0 in district A:
DEPOT1 -> ACM -> AES -> AFY -> AHC -> AIE -> AKO -> AKM -> APB -> ASN -> AMU -> ANY -> AOA -> ATS -> AUW -> DEPOT1
Total distance for vehicle 0 in district A: 89 units
----------
Analyzing delivery district: B
Optimized route for vehicle 0 in district B:
DEPOT1 -> ACO -> AAH -> ADU -> API -> AVC -> AWG -> AWF -> AWD -> DEPOT1
Total distance for vehicle 0 in district B: 73 units
----------
Analyzing delivery district: C
Optimized route for vehicle 0 in district C:
DEPOT1 -> AAK -> AAO -> AFE -> AOJ -> AWJ -> AWI -> AUA -> APK -> ANE -> AKU -> AIM -> DEPOT1
Total distance for vehicle 0 in district C: 114 units
----------
Analyzing delivery district: D
Optimized route for vehicle 0 in district D:
DEPOT1 -> AAP -> AAT -> AFF -> AGN -> AHR -> AIT -> AJX -> ANJ -> ANL -> APT -> AQX -> ATF -> AWQ -> AUF -> ATB -> ANH -> DEPOT1
Total distance for vehicle 0 in district D: 125 units
----------
A

### Populate master_routes_dict
<a id='sheesh'></a>

In [564]:
def populate_master_dict_routes(df):
    """Populates the master_dict_route"""
    result = {}
    for index, row in df.iterrows():
        paths = row['paths']
        path_lengths = row['path_lengths']
        if isinstance(paths, str):
            paths = ast.literal_eval(paths)
        if isinstance(path_lengths, str):
            path_lengths = ast.literal_eval(path_lengths)
        result[index] = {
            'paths': paths,
            'path_lengths': path_lengths
        }
    return result

print(type(routes_per_district))

temp_ = routes_per_district.copy()
temp_ = temp_.T # Transpose df
pprint(temp_)
master_routes_dict = populate_master_dict_routes(temp_)
del temp_
print(master_routes_dict)

#print(master_dict_routes)

print(master_routes_dict["A"]["paths"][0])


<class 'pandas.core.frame.DataFrame'>
                                               paths path_lengths
A  {0: ['DEPOT1', 'ACM', 'AES', 'AFY', 'AHC', 'AI...      {0: 89}
B  {0: ['DEPOT1', 'ACO', 'AAH', 'ADU', 'API', 'AV...      {0: 73}
C  {0: ['DEPOT1', 'AAK', 'AAO', 'AFE', 'AOJ', 'AW...     {0: 114}
D  {0: ['DEPOT1', 'AAP', 'AAT', 'AFF', 'AGN', 'AH...     {0: 125}
E  {0: ['DEPOT1', 'AZL', 'BCY', 'BEC', 'BFI', 'BH...     {0: 125}
F  {0: ['DEPOT1', 'BDB', 'BAT', 'BQX', 'BQZ', 'BO...     {0: 123}
G  {0: ['DEPOT1', 'BDG', 'BFR', 'BCF', 'BKH', 'BJ...     {0: 110}
H  {0: ['DEPOT1', 'AYW', 'AYY', 'BAC', 'BCJ', 'BE...     {0: 146}
I  {0: ['DEPOT1', 'AAY', 'ADI', 'ADK', 'AIA', 'AJ...     {0: 286}
{'A': {'paths': {0: ['DEPOT1', 'ACM', 'AES', 'AFY', 'AHC', 'AIE', 'AKO', 'AKM', 'APB', 'ASN', 'AMU', 'ANY', 'AOA', 'ATS', 'AUW', 'DEPOT1']}, 'path_lengths': {0: 89}}, 'B': {'paths': {0: ['DEPOT1', 'ACO', 'AAH', 'ADU', 'API', 'AVC', 'AWG', 'AWF', 'AWD', 'DEPOT1']}, 'path_lengths': {0: 73}}, 'C': {'path

## Visualize delivery route

In [565]:
export_map_to_excel_with_formatting(map_parcel, excel_pathvis_wb, excel_pathvis_ws)

In [566]:
# TODO visualizes

# Create milkruns for autonomous delivery

## Define sum function

Every point has to be visited once.
The delivery vehicle has to go from the depot to the point and return (depending on the carry capacity)
Distance per house = <distance_from_depot_to_node> * 2

In [567]:
def calculate_runs_per_house(lot_name, carry_capacity: int = 1):
    parcel_qty = master_houses_dict[lot_name][key_h_parcels]
    runs = math.ceil(parcel_qty / carry_capacity)
    master_houses_dict[lot_name][key_h_delivery_runs_required] = runs
    return runs


In [568]:
def calculate_milk_run_distances(df: pd.DataFrame, district_name, carry_capacity:int = 1):
    """
    Calculate the sum of all values in row 0 of a DataFrame.
    :param df:
    :return:
    """
    headers = list(df)
    # Filter Depots from list
    filtered_headers = [col for col in headers if name_string_depot not in col]
    row = df.iloc[0]
    row_values = row.tolist()
    #print(filtered_headers)
    # Get quantities for each lot

    path_length = 0
    for index, lot_name in enumerate(headers):
        if lot_name in filtered_headers:
            path_length_per_lot = 0
            run_multiplier = calculate_runs_per_house(lot_name, carry_capacity)
            distance = row_values[index]
            distance_both_ways = distance * 2
            distance_all_runs = distance_both_ways * run_multiplier
            path_length_per_lot += distance_all_runs
            print(f"[{district_name}]Path length for {lot_name}= {path_length_per_lot} Parcels={master_houses_dict[lot_name][key_h_parcels]}, Distance from Depot={row_values[index]}")
            path_length += path_length_per_lot
    #print(row.tolist())
    return path_length


In [569]:
print(distance_matrices["A"])
print(calculate_milk_run_distances(distance_matrices["A"], "A"))

        DEPOT1  ACM  AES  AFY  AHC  AIE  AKM  AKO  AMU  ANY  AOA  APB  ASN   
DEPOT1       0    5    7    8    9   10   12   12   14   15   15   19   22  \
ACM          5    0    5    6    7    8   10   10   12   13   13   17   20   
AES          7    5    0    4    5    6    8    8   10   11   11   15   18   
AFY          8    6    4    0    2    5    7    7    9   10   10   14   17   
AHC          9    7    5    2    0    4    6    6    8    9    9   13   16   
AIE         10    8    6    5    4    0    5    5    7    8    8   12   15   
AKM         12   10    8    7    6    5    0    3    3    6    6    8   11   
AKO         12   10    8    7    6    5    3    0    5    6    6   10   13   
AMU         14   12   10    9    8    7    3    5    0    2    4    8   11   
ANY         15   13   11   10    9    8    6    6    2    0    3   11   14   
AOA         15   13   11   10    9    8    6    6    4    3    0   11   14   
APB         19   17   15   14   13   12    8   10    8   11   11

In [570]:
def populate_autonomous_delivery_dict():
    for district, distance_matrix in distance_matrices.items():
        master_autonomous_dict[district] = {}
        print(f"Calculating milk run distances for district {district}")
        distance = calculate_milk_run_distances(df=distance_matrix, district_name=district, carry_capacity=par_sys_aut_vehicle_capacity)
        master_autonomous_dict[district][key_a_distance] = distance
    print(master_autonomous_dict)


In [571]:
populate_autonomous_delivery_dict()

Calculating milk run distances for district A
[A]Path length for ACM= 10 Parcels=1, Distance from Depot=5
[A]Path length for AES= 42 Parcels=3, Distance from Depot=7
[A]Path length for AFY= 96 Parcels=6, Distance from Depot=8
[A]Path length for AHC= 18 Parcels=1, Distance from Depot=9
[A]Path length for AIE= 60 Parcels=3, Distance from Depot=10
[A]Path length for AKM= 24 Parcels=1, Distance from Depot=12
[A]Path length for AKO= 24 Parcels=1, Distance from Depot=12
[A]Path length for AMU= 84 Parcels=3, Distance from Depot=14
[A]Path length for ANY= 90 Parcels=3, Distance from Depot=15
[A]Path length for AOA= 90 Parcels=3, Distance from Depot=15
[A]Path length for APB= 76 Parcels=2, Distance from Depot=19
[A]Path length for ASN= 88 Parcels=2, Distance from Depot=22
[A]Path length for ATS= 120 Parcels=3, Distance from Depot=20
[A]Path length for AUW= 42 Parcels=1, Distance from Depot=21
Calculating milk run distances for district B
[B]Path length for AAH= 16 Parcels=1, Distance from Depot

# Model output

## Modifier layer
Apply defined multipliers to adjust parameters to approximate real-world values

In [572]:
# Copy all master dicts to not modify those
mod_master_houses_dict = copy.deepcopy(master_houses_dict)
mod_master_city_dict = copy.deepcopy(master_city_dict)
mod_master_routes_dict = copy.deepcopy(master_routes_dict)
mod_master_autonomous_dict = copy.deepcopy(master_autonomous_dict)

# Modify all distances
print("Applying modifier for distances")
for district in delivery_districts:
    # Modify conventional distances
    current_route_path_length = master_routes_dict[district][key_r_path_lengths][0]  # Todo add possibility to change vehicles
    modified_route_path_length = round(current_route_path_length * par_sys_distance_multiplier)
    print(f"District {district} (conventional) distance was: {current_route_path_length} | modified to: {modified_route_path_length}")
    mod_master_routes_dict[district][key_r_path_lengths][0] = modified_route_path_length

    # Modify autonomous distances
    current_autonomous_path_length = master_autonomous_dict[district][key_a_distance]
    modified_autonomous_path_length = round(current_autonomous_path_length * par_sys_distance_multiplier)
    print(f"District {district} (autonomous) distance was: {current_autonomous_path_length} | modified to: {modified_autonomous_path_length}")
    mod_master_autonomous_dict[district][key_a_distance] = modified_autonomous_path_length



Applying modifier for distances
District A (conventional) distance was: 89 | modified to: 67
District A (autonomous) distance was: 864 | modified to: 648
District B (conventional) distance was: 73 | modified to: 55
District B (autonomous) distance was: 516 | modified to: 387
District C (conventional) distance was: 114 | modified to: 86
District C (autonomous) distance was: 1008 | modified to: 756
District D (conventional) distance was: 125 | modified to: 94
District D (autonomous) distance was: 2080 | modified to: 1560
District E (conventional) distance was: 125 | modified to: 94
District E (autonomous) distance was: 1914 | modified to: 1436
District F (conventional) distance was: 123 | modified to: 92
District F (autonomous) distance was: 828 | modified to: 621
District G (conventional) distance was: 110 | modified to: 82
District G (autonomous) distance was: 542 | modified to: 406
District H (conventional) distance was: 146 | modified to: 110
District H (autonomous) distance was: 147

## Gather output

In [573]:
def populate_master_city_dict():
    number_of_houses = 0
    number_of_households = 0
    number_of_parcels = 0
    number_of_redirected_parcels = 0
    number_of_stops = 0

    for lot, info in master_houses_dict.items():
        if info[key_h_lot_use] == map_char_house:
            number_of_houses += 1
        if info[key_h_lot_use] == map_char_parcel:
            number_of_stops += 1
        number_of_households += info[key_h_households]
        number_of_parcels += info[key_h_parcels]
        number_of_redirected_parcels += info[key_h_parcels_redirected]

    master_city_dict[key_c_houses] = number_of_houses
    master_city_dict[key_c_households] = number_of_households
    master_city_dict[key_c_total_parcels] = number_of_parcels
    master_city_dict[key_c_total_stops] = number_of_stops
    master_city_dict[key_c_total_parcels_redirected] = number_of_redirected_parcels

In [574]:
def populate_master_model_output_dict():
    parcels_by_district = {}
    for district in delivery_districts:
        master_model_output_dict[district] = {}
        total_parcels_in_district = 0
        total_stops_in_district = 0
        total_redirected_parcels_in_district = 0

        # Sum total number of stops in district
        for lot, info in master_houses_dict.items():
            # print("lot", lot, "info",info)
            if info[key_h_parcels] > 0 and info[key_h_delivery_district]==district:
                total_parcels_in_district += int(info[key_h_parcels])
                total_stops_in_district += 1

        # Sum total redirected parcels in district
        for lot, info in master_houses_dict.items():
            redirected_parcels = info[key_h_parcels_redirected]
            if redirected_parcels  > 0 and info[key_h_delivery_district]==district:
                total_redirected_parcels_in_district += redirected_parcels

        master_model_output_dict[district][key_o_parcels] = total_parcels_in_district
        master_model_output_dict[district][key_o_parcels_redirected] = total_redirected_parcels_in_district
        master_model_output_dict[district][key_o_stops] = total_stops_in_district
        master_model_output_dict[district][key_o_drop_factor] = 0 if total_stops_in_district == 0 else (total_parcels_in_district / total_stops_in_district)
        master_model_output_dict[district][key_o_distance_conventional] = master_routes_dict[district][key_r_path_lengths][0]
        master_model_output_dict[district][key_o_distance_autonomous] = master_autonomous_dict[district][key_a_distance]


In [575]:
populate_master_model_output_dict()
populate_master_city_dict()

print("District data:")
pprint(master_model_output_dict)

print("------")

print("City data:")
pprint(master_city_dict)

District data:
{'A': {'dist_auto': 864,
       'dist_conv': 89,
       'drop_factor': 2.357142857142857,
       'parcels': 33,
       'parcels_redirected': 3,
       'stops': 14},
 'B': {'dist_auto': 516,
       'dist_conv': 73,
       'drop_factor': 1.5,
       'parcels': 12,
       'parcels_redirected': 1,
       'stops': 8},
 'C': {'dist_auto': 1008,
       'dist_conv': 114,
       'drop_factor': 1.9090909090909092,
       'parcels': 21,
       'parcels_redirected': 2,
       'stops': 11},
 'D': {'dist_auto': 2080,
       'dist_conv': 125,
       'drop_factor': 2.25,
       'parcels': 36,
       'parcels_redirected': 2,
       'stops': 16},
 'E': {'dist_auto': 1914,
       'dist_conv': 125,
       'drop_factor': 1.7857142857142858,
       'parcels': 25,
       'parcels_redirected': 1,
       'stops': 14},
 'F': {'dist_auto': 828,
       'dist_conv': 123,
       'drop_factor': 1.25,
       'parcels': 10,
       'parcels_redirected': 0,
       'stops': 8},
 'G': {'dist_auto': 542,
   